# Problem description

You've escaped Commander Lambda's exploding space station along with numerous escape pods full of bunnies. But - oh no! - one of the escape pods has flown into a nearby nebula, causing you to lose track of it. You start monitoring the nebula, but unfortunately, just a moment too late to find where the pod went. However, you do find that the gas of the steadily expanding nebula follows a simple pattern, meaning that you should be able to determine the previous state of the gas and narrow down where you might find the pod.

From the scans of the nebula, you have found that it is very flat and distributed in distinct patches, so you can model it as a 2D grid. You find that the current existence of gas in a cell of the grid is determined exactly by its 4 nearby cells, specifically, (1) that cell, (2) the cell below it, (3) the cell to the right of it, and (4) the cell below and to the right of it. If, in the current state, exactly 1 of those 4 cells in the 2x2 block has gas, then it will also have gas in the next state. Otherwise, the cell will be empty in the next state.

For example, let's say the previous state of the grid (p) was:
.O..
..O.
...O
O...

To see how this grid will change to become the current grid (c) over the next time step, consider the 2x2 blocks of cells around each cell.  Of the 2x2 block of [p[0][0], p[0][1], p[1][0], p[1][1]], only p[0][1] has gas in it, which means this 2x2 block would become cell c[0][0] with gas in the next time step:
.O -> O
..

Likewise, in the next 2x2 block to the right consisting of [p[0][1], p[0][2], p[1][1], p[1][2]], two of the containing cells have gas, so in the next state of the grid, c[0][1] will NOT have gas:
O. -> .
.O

Following this pattern to its conclusion, from the previous state p, the current state of the grid c will be:
O.O
.O.
O.O

Note that the resulting output will have 1 fewer row and column, since the bottom and rightmost cells do not have a cell below and to the right of them, respectively.

Write a function solution(g) where g is an array of array of bools saying whether there is gas in each cell (the current scan of the nebula), and return an int with the number of possible previous states that could have resulted in that grid after 1 time step.  For instance, if the function were given the current state c above, it would deduce that the possible previous states were p (given above) as well as its horizontal and vertical reflections, and would return 4. The width of the grid will be between 3 and 50 inclusive, and the height of the grid will be between 3 and 9 inclusive.  The answer will always be less than one billion (10^9).


**Test cases**

Your code should pass the following test cases.

Note that it may also be run against hidden test cases not shown here.

Input: solution.solution([[True, True, False, True, False, True, False, True, True, False], [True, True, False, False, False, False, True, True, True, False], [True, True, False, False, False, False, False, False, False, True], [False, True, False, False, False, False, True, True, False, False]])

Output: 11567


Input: solution.solution([[True, False, True], [False, True, False], [True, False, True]])

Output: 4

Input: solution.solution([[True, False, True, False, False, True, True, True], [True, False, True, False, False, False, True, False], [True, True, True, False, False, False, True, False], [True, False, True, False, False, False, True, False], [True, False, True, False, False, True, True, True]])
Output: 254

# Introduction
This problem can be modeled as inverting a cellular automaton, counting the number of possible previous grids that could have originated the actual grid. 

Celullar automata is a model that envolves through time by sucessive appling a rule to each cell of a map, taking as input the value of that cell and of the neighbours cells. The map of this problem is a 2D grid (the Nebula), the state is a boolean (have gas or not), the neighbourhood of the cell (x,y) are the cells {(x+1,y+1),(x+1,y),(x,y+1)}, and the transfer rule is given by:

| (x,y) | (x+1,y) | (x,y+1) | (x+1,y+1) | out  |
| ----- | ------- | ------- | --------- | ---- |
| 0     | 0       | 0       | 0         | 0    |
| 0     | 0       | 0       | 1         | 1    |
| 0     | 0       | 1       | 0         | 1    |
| 0     | 0       | 1       | 1         | 0    |
| 0     | 1       | 0       | 0         | 1    |
| 0     | 1       | 0       | 1         | 0    |
| 0     | 1       | 1       | 0         | 0    |
| 0     | 1       | 1       | 1         | 0    |
| 1     | 0       | 0       | 0         | 1    |
| 1     | 0       | 0       | 1         | 0    |
| 1     | 0       | 1       | 0         | 0    |
| 1     | 0       | 1       | 1         | 0    |
| 1     | 1       | 0       | 0         | 0    |
| 1     | 1       | 0       | 1         | 0    |
| 1     | 1       | 1       | 0         | 0    |
| 1     | 1       | 1       | 1         | 0    |

However, this automata is not reversible (aka not bijective), and there is no known way to directly find the number of possible previous states (if it was reversible there will be no point in counting opossibilities since there will be only one). I found two very different implementations to count by brute force, \cite{1} and \cite{2}. The two have approximately the same runtime (see table below for comparison).

| Algorithm  | Runtime |
| ---------- | ------- |
| Solution 1 | 2.233 s |
| Solution 2 | 2.267 s |

# References
[1] - Berto, Francesco and Tagliabue, Jacopo, "Cellular Automata", The Stanford Encyclopedia of Philosophy (Fall 2017 Edition), Edward N. Zalta (ed.). Available in: <https://plato.stanford.edu/archives/fall2017/entries/cellular-automata/>. 

[2] - Gauvain Tarbouriech. Personal Website. Available in: https://govanify.com/post/foobar/

[3] - Nk_mason. Answer in Mathematics Stack Exchange. Available in: https://math.stackexchange.com/a/2368089


In [132]:
# Implementation from \cite{3}
box = {
    ((0, 0), (0, 0)) : 0,
    ((0, 0), (0, 1)) : 1,
    ((0, 0), (1, 0)) : 1,
    ((0, 0), (1, 1)) : 0,
    ((0, 1), (0, 0)) : 1,
    ((0, 1), (0, 1)) : 0,
    ((0, 1), (1, 0)) : 0,
    ((0, 1), (1, 1)) : 0,
    ((1, 0), (0, 0)) : 1,
    ((1, 0), (0, 1)) : 0,
    ((1, 0), (1, 0)) : 0,
    ((1, 0), (1, 1)) : 0,
    ((1, 1), (0, 0)) : 0,
    ((1, 1), (0, 1)) : 0,
    ((1, 1), (1, 0)) : 0,
    ((1, 1), (1, 1)) : 0,
}



true = True
false = False


def get_col_comb(first,column):
    x = ((0,0),(0,1),(1,0),(1,1))
    count_possibility = []

    for key in first:
        nextCol = []

        for val in x:
            if box[((key[0],key[1]),val)] == column[0]:
                nextCol.append(val)

        for n in range(1,len(column)):
            newCol = []
            if len(nextCol) == 0:
                break
            for col in nextCol:
                for m in range(2):
                    tempCol = list(col)
                    if box[((key[n],key[n+1]),(col[n],m))] == column[n]:
                        tempCol.append(m)
                        newCol.append(tempCol)
            nextCol = newCol
        [count_possibility.append((key,tuple(c))) for c in nextCol]
    return tuple(count_possibility)

def swap_row_col(g):
    return tuple(zip(*g))

def nk(firstColumn):
    def first_col_int(f_c_col):
        x = ((0,0),(0,1),(1,0),(1,1))
        g0 = f_c_col[0]
        l = []
        for key,val in box.items():
            if g0 == val:
                l.append(key)
        present = tuple(l)
        for n in range(1,len(f_c_col)):
            new = []
            for z in present:
                for comb in x:
                    possibility = (z[n],comb)

                    if box[possibility] == f_c_col[n]:
                        temp = list(z)
                        temp.append(comb)
                        new.append(temp)

            present = tuple(new)
        return tuple([swap_row_col(x) for x in present])

    if firstColumn in no_col:
        return no_col[firstColumn]
    else:
        right_grids = first_col_int(firstColumn)
        no_col[firstColumn] = right_grids
        return right_grids

no_col = {}

def solution2(g):
    rotation = swap_row_col(g)
    first = {}
    right_grids = nk(rotation[0])
    for row in right_grids: print (row)
    for z in right_grids:
        if z[1] not in first:
            first[z[1]] = 1
        else:
            first[z[1]] = first[z[1]] + 1
    for n in range(1,len(rotation)):
        second = {}
        newGrids = get_col_comb(first,rotation[n])
        total = len(newGrids)
        for z in newGrids:
            isValid_counter = 0
            if z[0] in first:
                isValid_counter += 1
                if z[1] in second:
                    second[z[1]] = first[z[0]] + second[z[1]]
                else:
                    second[z[1]] = first[z[0]]
        first = second
    meter = 0
    for row, count in first.items():
        meter += count
    return meter

In [133]:
# Implementation from \cite{2}
def solution(state, a=0, b=0, past=0, solutions=0, history=0):
    if(past==0):
        past=[[True] * (len(state[0])+1) for i in range(len(state)+1)]
        solutions = {}
        history = []

    if(b==len(state[0])+1):
        return True 

    res=0
    index=((a,b), tuple(history[-(len(state)+2):]))
    if index in solutions:
        return solutions[index]

    for cell in [True, False]:
        if (not a or not b) or len(set([((past[a][b-1] + past[a-1][b]
            + past[a-1][b-1] + cell)==1), state[a-1][b-1]]))==1:
                history.append(cell)
                past[a][b] = cell
                res+=solution(state, a=(a+1)%(len(state)+1),
                        b=b+(a+1)//(len(state)+1), past=past,
                        solutions=solutions, history=history)
                history.pop()
    solutions[index]=res
    return res

In [96]:
from time import time
import gc

X = [
        ([[True, True, False, True, False, True, False, True, True, False], [True, True, False, False, False, False, True, True, True, False], [True, True, False, False, False, False, False, False, False, True], [False, True, False, False, False, False, True, True, False, False]]),
        ([[True, False, True], [False, True, False], [True, False, True]])
    ]*1000

y = [11567, 4]*1000
gc.collect()

for f in [solution2, solution]:
    time0 = time()
    y_hat = [solution2(arg) for arg in X]
    time1 = time()
    if y_hat != y: print ("Fail, dumb ass")
    else: print("%s ms" % ((time1 - time0)*1000))
    

2267.315149307251 ms
2233.9468002319336 ms
